In [1]:
import os
import re
import pandas as pd
import pickle
import collections
from collections import defaultdict
import numpy as np
import math
from ast import literal_eval
from time import gmtime, strftime
import re
import time
from bs4 import BeautifulSoup
from tqdm.auto import tqdm

# Scrapping
import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from fake_useragent import UserAgent

# Error Handling
import socket
import urllib3
import urllib.request
from urllib.request import urlopen
from urllib.parse import quote_plus
from urllib.request import urlretrieve
from selenium.common.exceptions import NoSuchElementException, ElementNotInteractableException, ElementClickInterceptedException
import warnings
warnings.filterwarnings('ignore')

In [2]:
wd = webdriver.Chrome(ChromeDriverManager().install())

wd.get('http://www.ssg.com/monm/brand.ssg')
time.sleep(2)

wd.maximize_window()

wd.find_element_by_xpath("/html/body/div[4]/div[6]/div/div[2]/div[2]/div[2]/div[1]/button[2]/span[1]").click()
time.sleep(1)

html = wd.page_source
soup = BeautifulSoup(html, "html.parser")

#soup.find_all("a", {"class": "brandSrch clickable"})



====== WebDriver manager ======
Current google-chrome version is 94.0.4606
Get LATEST driver version for 94.0.4606
Driver [/Users/kimjisoo/.wdm/drivers/chromedriver/mac64/94.0.4606.61/chromedriver] found in cache


In [3]:
#brand id set

brd_lst = soup.find_all("a", {"class": "brandSrch clickable"})

brd_id_lst = []
for brd in brd_lst:
    brd_id_lst.append(re.findall('data-brand-id="(.*?)"', str(brd))[0])
    
brd_id_lst = list(set(brd_id_lst))

In [ ]:
import pickle

with open("ssg_brd_id_lst.txt", 'wb') as f:
    brd_id_lst = pickle.load(f)

In [8]:
all_brd_name = []
all_brd_id = []
all_prd_total_cnt = []
exception = []

In [9]:
#ssg_monm brand의 이름과 전체 상품 개수 수집

for brd_id in tqdm(brd_id_lst[:3]):
    
    options = Options()
    ua = UserAgent()
    userAgent = ua.random
    print(userAgent)

    options.add_argument('headless')
    options.add_argument('window-size=1920x1080')
    options.add_argument("disable-gpu")
    options.add_argument(f'user-agent={userAgent}')
    wd = webdriver.Chrome(ChromeDriverManager().install(), chrome_options=options)

    socket.setdefaulttimeout(30)
    
    
    try:
        wd.get("http://www.ssg.com/disp/brandShop?brandId={}".format(brd_id))
        time.sleep(3)

        html = wd.page_source
        soup = BeautifulSoup(html, "html.parser")

        brd_name = wd.find_element_by_xpath("/html/body/div[4]/div[6]/div[2]/div[1]/h2/a").text

        prd_total_cnt = soup.find("em", {"id": "area_item_total_count"}).text.replace('\t', '')
        prd_total_cnt = prd_total_cnt.replace('\n', '')
        prd_total_cnt = int(prd_total_cnt.replace(',', ''))


    except:
        brd_name = None
        brd_id = brd_id
        prd_total_cnt = None
        print("에러가 발생했습니다.")

    wd.close()
    
    all_brd_name.append(brd_name)
    all_brd_id.append(brd_id)
    all_prd_total_cnt.append(prd_total_cnt)

  0%|          | 0/3 [00:00<?, ?it/s]



====== WebDriver manager ======
Current google-chrome version is 94.0.4606
Get LATEST driver version for 94.0.4606
Driver [/Users/kimjisoo/.wdm/drivers/chromedriver/mac64/94.0.4606.61/chromedriver] found in cache


Mozilla/5.0 (Macintosh; Intel Mac OS X 10_7_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/27.0.1453.93 Safari/537.36




====== WebDriver manager ======
Current google-chrome version is 94.0.4606
Get LATEST driver version for 94.0.4606
Driver [/Users/kimjisoo/.wdm/drivers/chromedriver/mac64/94.0.4606.61/chromedriver] found in cache


Mozilla/5.0 (Windows NT 10.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/40.0.2214.93 Safari/537.36




====== WebDriver manager ======
Current google-chrome version is 94.0.4606
Get LATEST driver version for 94.0.4606


Mozilla/5.0 (Windows NT 6.2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/28.0.1467.0 Safari/537.36


Driver [/Users/kimjisoo/.wdm/drivers/chromedriver/mac64/94.0.4606.61/chromedriver] found in cache


In [10]:
#dataframe

ssg_brd_df = pd.DataFrame(columns = ['brd_name', 'brd_id', 'prd_total_cnt'])

ssg_brd_df['brd_name'] = all_brd_name
ssg_brd_df['brd_id'] = all_brd_id
ssg_brd_df['prd_total_cnt'] = all_prd_total_cnt

In [11]:
ssg_brd_df

,brd_name,brd_id,prd_total_cnt
0,키스뉴욕,3000029429,187
1,LA VIGUEUR,1010770896,9
2,비페스타,3000010658,35
